In [15]:
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai.chat_models import ChatGoogleGenerativeAI
from typing import List, Annotated, Literal, TypedDict
import operator

# ================================
# Dummy Tool: get_weather
# ================================
@tool
def get_weather(location: str):
    """Trả về thời tiết hiện tại ở một địa điểm."""
    return f"The current weather in {location} is sunny and 30°C."

tools = [get_weather]

# ================================
# State Definition
# ================================
class SimpleState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]

# ================================
# Model
# ================================
model = ChatGoogleGenerativeAI(model='gemini-1.5-flash', temperature=0).bind_tools(tools=tools)

# ================================
# Node: Agent
# ================================
def agent_node(state: SimpleState):
    response = model.invoke(state['messages'])
    return {"messages": [response]}

# ================================
# Node: Ask human (for clarification)
# ================================
def ask_human(state: SimpleState):
    print(f"🤖 AI: {state['messages'][-1].content}")
    user_input = input("👤 You: ")
    return {"messages": [HumanMessage(content=user_input)]}

# ================================
# Router Logic
# ================================
def router(state: SimpleState) -> Literal["tools", "ask", "end"]:
    last_msg = state["messages"][-1]
    if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
        return "tools"
    elif isinstance(last_msg, AIMessage):
        return "ask"
    else:
        return "end"

# ================================
# Build LangGraph
# ================================
graph = StateGraph(SimpleState)

graph.set_entry_point("agent")
graph.add_node("agent", agent_node)
graph.add_node("tools", ToolNode(tools))
graph.add_node("ask", ask_human)

graph.add_conditional_edges("agent", router, {
    "tools": "tools",
    "ask": "ask",
    "end": END
})

graph.add_edge("tools", "agent")
graph.add_edge("ask", "agent")

# ================================
# Compile App
# ================================
app = graph.compile()

# ================================
# Run Demo
# ================================
if __name__ == "__main__":
    print("🔁 Simple Human-in-the-loop Chatbot (weather/greeting)")
    user_input = input("👤 You: ")
    state = {"messages": [HumanMessage(content=user_input)]}
    final_state = app.invoke(state)

    print("\n✅ Final messages:")
    for m in final_state["messages"]:
        print(f"{m.type.upper()}: {m.content}")


🔁 Simple Human-in-the-loop Chatbot (weather/greeting)


AttributeError: Unknown field for Part: thought